In [ ]:
import torchvision.models as models

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip '/content/gdrive/My Drive/11785/hw2p2/11-785-fall-20-homework-2-part-2.zip'

Streaming output truncated to the last 5000 lines.
  inflating: verification_data/00064100.jpg  
  inflating: verification_data/00064101.jpg  
  inflating: verification_data/00064102.jpg  
  inflating: verification_data/00064103.jpg  
  inflating: verification_data/00064104.jpg  
  inflating: verification_data/00064105.jpg  
  inflating: verification_data/00064106.jpg  
  inflating: verification_data/00064107.jpg  
  inflating: verification_data/00064108.jpg  
  inflating: verification_data/00064109.jpg  
  inflating: verification_data/00064110.jpg  
  inflating: verification_data/00064111.jpg  
  inflating: verification_data/00064112.jpg  
  inflating: verification_data/00064113.jpg  
  inflating: verification_data/00064114.jpg  
  inflating: verification_data/00064115.jpg  
  inflating: verification_data/00064116.jpg  
  inflating: verification_data/00064117.jpg  
  inflating: verification_data/00064118.jpg  
  inflating: verification_data/00064119.jpg  
  inflating: verification_dat

In [ ]:
%matplotlib inline
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
from PIL import Image
import torch
from torch.autograd import Variable
import PIL.ImageOps    
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [ ]:
folder_dataset = torchvision.datasets.ImageFolder(root='/content/classification_data/train_data', 
                                                       transform=transforms.Compose([transforms.Resize((224,224)),
                                                                      transforms.ToTensor() ])) #,transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 #std=[0.229, 0.224, 0.225])
                                                                      
ImageFolder_dataloader = DataLoader(folder_dataset, batch_size=200, shuffle=True, num_workers=8)
folder_dataset.__len__(), len(folder_dataset.classes)

(380638, 4000)

In [ ]:
dev_dataset = torchvision.datasets.ImageFolder(root='/content/classification_data/val_data', 
                                               transform=transforms.Compose([transforms.Resize((224,224)),
                                                                      transforms.ToTensor() ])) #,transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 #std=[0.229, 0.224, 0.225])
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=200, 
                                             shuffle=False, num_workers=8)

RESNET 18

In [ ]:
#reference taken from https://github.com/pytorch/vision/blob/master/torchvision/models
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out




In [ ]:
#reference taken from https://github.com/pytorch/vision/blob/master/torchvision/models
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=4000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)


In [ ]:

model=ResNet( BasicBlock, [2,2,2,2])
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
resnet=models.resnet18(pretrained=False)

resnet18=nn.Sequential(resnet, nn.Linear(1000,4000, bias=True))
print(resnet18)

In [ ]:
numEpochs = 20
#num_feats = 3

learningRate = 0.15
weightDecay = 5e-5

#hidden_sizes = [32, 64]


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:



criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)

for loading checkpoint

In [ ]:
resnet18.load_state_dict(torch.load('/content/modelresnet18.pth'))

<All keys matched successfully>

training

In [ ]:
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
traindatalen=folder_dataset.__len__()
valdatalen=dev_dataset.__len__()
num_epochs=12
#model=resnet18
ypred=[]
ytrue=[]
scheduler = StepLR(optimizer, step_size=1, gamma=0.85)
print(model)
cuda = True if torch.cuda.is_available() else False
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
#loss = nn.CrossEntropyLoss()

if cuda:
    model = model.cuda()
        #discriminator = discriminator.cuda()
    criterion.cuda()
#optimizer = torch.optim.Adam(model1.parameters(), lr=1e-4)
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
for epoch in range(num_epochs):
    correcttrain = 0
    correctval=0
    for i, batch in enumerate(ImageFolder_dataloader):
        model.train()
        X,Y=batch
        #print(X)
        #print(Y)
        #print(type(X))
        #print('x',X.shape)
        #print('y',Y.shape)
        #print(type(X.cuda()))
        #x = X.to('cuda')  #.cuda() #to(device='cuda')
        #y = Y.to('cuda')   #cuda() #to(device='cuda')
        x = Variable(X.type(Tensor)).to('cuda')
        y = Variable(Y.type(Tensor)).to('cuda')
        optimizer.zero_grad()
        output=model(x)
        print(output.shape)
        lossnet=criterion(output,y.long())
        lossnet.backward()
        #removing lr scheduler
        optimizer.step()
        _,output=torch.max(output,dim=1)
        correcttrain +=(output == y).float().sum()
        if epoch==(num_epochs-1):
            torch.save(model.state_dict(), "modelresnet.pth")
        print("Epoch {}/{}, Train Loss: {:.3f}".format(epoch+1,num_epochs, lossnet.item()))
        #print('output',output.shape)
        #print('target',y.shape)
    for i, batch in enumerate(dev_dataloader):
        model.eval()
        X,Y=batch
        #print('x',X.shape)
        #print('y',Y.shape)
        x = Variable(X.type(Tensor))
        y = Variable(Y.type(Tensor))
        outputval=model(x)    
        lossnetval=criterion(outputval,y.long())
        _,output=torch.max(outputval,dim=1)
        #Accuracy
        #output = (output>0.5).float()
        if(epoch==(num_epochs-1)):
          for i in range(len(y)):
              ytrue.append(y[i].item())
              ypred.append(output[i].item())
        correctval +=(output == y).float().sum()
        print("Epoch {}/{}, validation Loss: {:.3f}".format(epoch+1,num_epochs, lossnetval.item()))
    scheduler.step()
    accuracytrain = 100 * correcttrain / traindatalen
    accuracyval = 100 * correctval / valdatalen
    print("Train Accuracy = {}, Validation Accuracy = {}".format(accuracytrain, accuracyval))
#test data

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

KeyboardInterrupt: ignored

In [ ]:
torch.save(resnet18.state_dict(), '/content/modelresnet18.pth')

In [ ]:
print(folder_dataset.__len__())

380638


In [ ]:
print(len(dev_dataset.classes))

4000


function for validation and testing

In [ ]:
import os
from os import listdir
from os.path import isfile, join
data_dir = '/content/verification_pairs_test.txt'
path='/content/'
train_files = open(data_dir).read().splitlines()
transform=transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()])
def pairing():
    img1,img2,label=[],[],[]
    for i in range(len(train_files)):
        img1.append(transform(Image.open(join(path,train_files[i].split()[0]))))
        img2.append(transform(Image.open(join(path,train_files[i].split()[1]))))
        #l=train_files[i].split()[2]
        #label.append(torch.from_numpy(np.array(int(l), dtype='int32')))
    return img1,img2,label  
#s=train_files[10]
#c=s.split()[2]
#print(torch.from_numpy(np.array(int(c), dtype='int32')))
#train_labels = np.loadtxt(join(data_dir, 'train_labels.txt'), np.int32)
#data_files = [f for f in data_dir]
#print(train_files)




In [ ]:
print(accuracyval)

tensor(61.3875, device='cuda:0')


In [ ]:
from sklearn.metrics import roc_auc_score
ytrue=[]
ypred=[]
img1,img2=pairing() #,label

In [ ]:
x0=torch.unsqueeze(img1[1],0)
print(len(label))

8805


In [ ]:
validation and testing using above function

In [ ]:
for i in range(len(img1)):
    print(i)
    x0,x1,label2 = img1[i],img2[i],label[i]
    #print(x0.shape)
    x0=torch.unsqueeze(x0,0)
    x1=torch.unsqueeze(x1,0)
    #concatenated = torch.cat((x0,x1),0)
    #print(x0.shape)
    output1 = resnet18(Variable(x0)) #.cuda()
    output2 = resnet18(Variable(x1))
    #cos = F.cosine_similarity(output1, output2,dim=1, eps=1e-6)
    #cosine = cos(output1, output2)
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    cosine = cos(output1, output2)
    #print(cosine)
    
    ytrue.append(label2.item())
    ypred.append(cosine.item())
print(roc_auc_score(ytrue, ypred))

In [ ]:
print(roc_auc_score(ytrue, ypred))

0.8011359105507024


dataset class for validation and testing

In [ ]:
import os
from os import listdir
from os.path import isfile, join
data_dir = '/content/verification_pairs_test.txt'
path='/content/'
id=open(data_dir).read().splitlines()
pred=[]
ytrue=[]
class PairDataset(Dataset):
    def __init__(self, datapath,transform):
        self.train_files = open(data_dir).read().splitlines()
        self.transform=transform
    def __len__(self):
        return(len(self.train_files))    
    def __getitem__(self, idx):    
        img1=self.transform(Image.open(join(path,self.train_files[idx].split()[0])))
        img2=self.transform(Image.open(join(path,self.train_files[idx].split()[1])))
        #uncomment below and return same for validation. keeping commented for testing
        #l=self.train_files[idx].split()[2]
        #label=torch.from_numpy(np.array(int(l), dtype='int32'))
        return img1,img2

In [ ]:
transform=transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor() ]) 
test_dataloader = torch.utils.data.DataLoader(PairDataset(data_dir,transform), batch_size=1, shuffle=False, num_workers=8)

validation and testing with above class PairDataset

In [ ]:
for i, batch in enumerate(test_dataloader):
    x0,x1=batch
    print(i)
    print(x0.shape)
    
    output1 = model(Variable(x0).cuda()) #
    output2 = model(Variable(x1).cuda())
    #cos = F.cosine_similarity(output1, output2,dim=1, eps=1e-6)
    #cosine = cos(output1, output2)
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    cosine = cos(output1, output2)
    #print(cosine)
    #ytrue.append(l.item())
    pred.append(cosine.item())

Streaming output truncated to the last 5000 lines.
torch.Size([1, 3, 224, 224])
49336
torch.Size([1, 3, 224, 224])
49337
torch.Size([1, 3, 224, 224])
49338
torch.Size([1, 3, 224, 224])
49339
torch.Size([1, 3, 224, 224])
49340
torch.Size([1, 3, 224, 224])
49341
torch.Size([1, 3, 224, 224])
49342
torch.Size([1, 3, 224, 224])
49343
torch.Size([1, 3, 224, 224])
49344
torch.Size([1, 3, 224, 224])
49345
torch.Size([1, 3, 224, 224])
49346
torch.Size([1, 3, 224, 224])
49347
torch.Size([1, 3, 224, 224])
49348
torch.Size([1, 3, 224, 224])
49349
torch.Size([1, 3, 224, 224])
49350
torch.Size([1, 3, 224, 224])
49351
torch.Size([1, 3, 224, 224])
49352
torch.Size([1, 3, 224, 224])
49353
torch.Size([1, 3, 224, 224])
49354
torch.Size([1, 3, 224, 224])
49355
torch.Size([1, 3, 224, 224])
49356
torch.Size([1, 3, 224, 224])
49357
torch.Size([1, 3, 224, 224])
49358
torch.Size([1, 3, 224, 224])
49359
torch.Size([1, 3, 224, 224])
49360
torch.Size([1, 3, 224, 224])
49361
torch.Size([1, 3, 224, 224])
49362
torc

In [ ]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(ytrue, pred))

0.9323752796868024


In [ ]:
print(len(pred))

51835


generate result submission file

In [ ]:
import pandas as pd
pd.DataFrame({}).to_csv("result_1.csv")

In [ ]:
import csv
with open('/content/result_1.csv', 'w') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(("Id", "Category"))
    rcount = 0
    for row in id:
        wr.writerow((id[rcount], pred[rcount]))
        rcount = rcount + 1
    myfile.close()